# 01 — Transactions Data Cleaning

## 1.0 Business Understanding
Dormancy in customer accounts leads to reduced activity, low digital adoption, and lost business opportunities.  
To address this, we start by preparing a **clean transactions dataset** covering Dec 2024 – Aug 2025.  

This notebook focuses on:
- Merging raw monthly transaction reports.  
- Excluding invalid statuses (D, J, R).  
- Deriving reporting months from transaction dates.  
- Classifying transactions into channel groups.  
- Saving a clean file for later aggregation.

---

## 2.0 Data Sources
- **Transactions Data (Dec 2024 – Aug 2025)**  
  - Location: `data/raw/transactions/`  
  - Format: monthly `.csv` / `.xlsx` files.  
- **Exclusions**  
  - Status codes: D = Declined, J = Reversed, R = Rejected.  
- **Confidentiality**  
  - Raw files are excluded from GitHub (`.gitignore`).


In [ ]:
# 3.0 Import Libraries
import pandas as pd
import numpy as np
from pathlib import Path
import os

pd.set_option("display.max_columns", None)

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

## 4.0 Locate Files
We check both relative paths (`data/raw/transactions` and `../data/raw/transactions`)  
so the notebook works whether run from the project root or the `notebooks/` folder.

In [3]:
# Check current working directory
print("cwd:", os.getcwd())

# Try possible paths
p1 = Path("data/raw/transactions")
p2 = Path("../data/raw/transactions")

if p1.exists():
    raw_dir = p1
elif p2.exists():
    raw_dir = p2
else:
    raise FileNotFoundError("❌ Could not find transactions folder. Check project structure.")

# List files
files = list(raw_dir.glob("*.csv")) + list(raw_dir.glob("*.xlsx")) + list(raw_dir.glob("*.xls"))
print("Looking in:", raw_dir.resolve())
print("Files found:", len(files))
for f in files:
    print(" -", f.name)

if not files:
    raise FileNotFoundError(f"No transaction files found in {raw_dir}")

cwd: C:\Users\USER\Documents\Personal Projects\Dormancy\notebooks
Looking in: C:\Users\USER\Documents\Personal Projects\Dormancy\data\raw\transactions
Files found: 11
 - Branch Activity - 01.08.25 - 20.08.25.csv
 - Branch Activity - 21.08.25 - 31.08.25.csv
 - Branch Activity - 30.06.25.csv
 - Branch Activity - June 2025.csv
 - Branch Activity - March 2025.csv
 - Branch Activity -28.02.25.csv
 - Branch Activity 30.04.25.csv
 - Branch Activity 31.01.25.csv
 - Branch Activity 31.05.25.csv
 - Branch_Activity - July 2025.csv
 - Branch Activity - 31.12.24.xlsx


## 5.0 Load and Merge Monthly Transactions
All files are merged into one DataFrame.  

In [ ]:
# 5.0 Load and Merge Monthly Transactions
# Function to read CSV or Excel
def read_file(f):
    if f.suffix.lower() in [".xlsx", ".xls"]:
        return pd.read_excel(f)
    return pd.read_csv(f)

# Merge all transaction files
tx = pd.concat([read_file(f) for f in files], ignore_index=True)

# Clean column names
tx.columns = [c.strip() for c in tx.columns]

print("Merged shape:", tx.shape)
tx.head()

## 6.0 Clean Data
Steps:
1. Exclude invalid statuses (D, J, R).  
2. Convert `created_date` into datetime.  
3. Create `Reporting_Month` = month-end date.  Full-time

In [ ]:
# 6.0 Clean Data
# Steps:
# 1. Exclude invalid statuses (D, J, R).
# 2. Convert created_date into datetime.
# 3. Create Reporting_Month = month-end date.

# 1) Exclude invalid statuses
tx['status'] = tx['status'].astype(str).str.upper().str.strip()
tx = tx[~tx['status'].isin({'D','J','R'})]

# 2) Convert created_date
tx['created_date'] = pd.to_datetime(tx['created_date'], errors='coerce', dayfirst=True)

# 3) Reporting Month
tx['Reporting_Month'] = tx['created_date'].dt.to_period('M').dt.to_timestamp('M')

tx[['created_date','Reporting_Month']].head()

## 7.0 Classify Channels
Rules:
- **Mobile** → tran_type in Mobile Banking (withdrawal, transfer, charges, airtime, utility, paybill, pesalink).  
- **Biz2Bank** → RTS PAYBILL DEPOSITS, RTS MPESA DEPOSIT.  
- **GABCollect** → GABCollect.  
- **Internet** → if created_by = IBKTLR.  
- **ATM** → if channels = ATM.  
- **Other** → everything else.

In [ ]:
# 7.0 Classify Channels

MOBILE_TYPES = {
    'MOBILE BANKING MPESA WITHDRAWAL',
    'MOBILE BANKING TRANSFER',
    'MB CHARGES',
    'MOBILE BANKING AIRTIME PURCHASE',
    'MB UTILITY PAYMENT',
    'MB PAYBILL',
    'PESALINK TRANSFER',
    'PESALINK DEPOSIT'
}
BIZ2BANK_TYPES = {'RTS PAYBILL DEPOSITS', 'RTS MPESA DEPOSIT'}

def classify_channel(row):
    tt = str(row.get('tran_type','')).upper().strip()
    created_by = str(row.get('created_by','')).upper().strip()
    ch = str(row.get('channels','')).upper().strip()
    
    if tt in MOBILE_TYPES or tt.startswith("MOBILE BANKING") or "PESALINK" in tt:
        return "Mobile"
    if tt in BIZ2BANK_TYPES:
        return "Biz2Bank"
    if tt == "GABCOLLECT":
        return "GABCollect"
    if created_by == "IBKTLR":
        return "Internet"
    if ch == "ATM":
        return "ATM"
    return "Other"

# Apply classification
tx['Channel_Group'] = tx.apply(classify_channel, axis=1)

# Flags
tx['IsDigital'] = tx['Channel_Group'].isin(['Mobile','Internet','Biz2Bank']).astype(int)
tx['IsPhysical'] = tx['Channel_Group'].eq('ATM').astype(int)

tx['Channel_Group'].value_counts()

## 8.0 Save Clean Transactions
The cleaned dataset is saved (ignored by Git).

In [ ]:
# 8.0 Save Clean Transactions
out_path = Path("data/processed/transactions_clean.csv")
tx.to_csv(out_path, index=False)
print("Clean data saved to:", out_path.resolve())

# 02 — Dormancy_Balances_Cleaning

## 1.0 Purpose
Balances data provides the monthly end balance per customer.  
We will clean this dataset, ensure proper account classification (Current, Savings, FD),  
and prepare it for integration with the transactions dataset.

---

## 2.0 Data Sources
- **Balances Data (Dec 2024 – Aug 2025)**  
  - Location: `data/raw/balances/`  
  - Format: `.csv` or `.xlsx`.  
- **GL Mapping**  
  - Provided mapping of `gl_name` → Account Type.  
- **Confidentiality**  
  - Raw files are excluded from GitHub (`.gitignore`).


In [ ]:
# 02 — Balances Data Cleaning

# 3.0 Import Libraries
import pandas as pd
from pathlib import Path
import os

pd.set_option("display.max_columns", None)

# 4.0 Locate Files
print("cwd:", os.getcwd())

p1 = Path("data/raw/balances")
p2 = Path("../data/raw/balances")

if p1.exists():
    bal_dir = p1
elif p2.exists():
    bal_dir = p2
else:
    raise FileNotFoundError("❌ Could not find balances folder. Check project structure.")

bal_files = list(bal_dir.glob("*.csv")) + list(bal_dir.glob("*.xlsx")) + list(bal_dir.glob("*.xls"))
print("Looking in:", bal_dir.resolve())
print("Files found:", len(bal_files))
for f in bal_files:
    print(" -", f.name)

if not bal_files:
    raise FileNotFoundError(f"No balance files found in {bal_dir}")

# 5.0 Load and Merge Monthly Balances
def read_file(f):
    if f.suffix.lower() in [".xlsx", ".xls"]:
        return pd.read_excel(f)
    return pd.read_csv(f)

balances = pd.concat([read_file(f) for f in bal_files], ignore_index=True)
balances.columns = [c.strip() for c in balances.columns]
print("Merged shape:", balances.shape)
balances.head()

# 6.0 Clean Data
for col in ['date_opened', 'birth_date', 'date_created']:
    if col in balances.columns:
        balances[col] = pd.to_datetime(balances[col], errors='coerce', dayfirst=True)

if 'Reporting_Month' not in balances.columns:
    if 'date_created' in balances.columns:
        balances['Reporting_Month'] = balances['date_created'].dt.to_period('M').dt.to_timestamp('M')

# 7.0 Map GL Accounts → Account Type
gl_mapping = {
    'CURRENT ACCOUNT': 'Current',
    'SAVINGS ACCOUNT': 'Savings',
    'FIXED DEPOSIT': 'FD'
}

if 'gl_name' in balances.columns:
    balances['Account_Type'] = balances['gl_name'].map(
        lambda x: gl_mapping.get(str(x).upper(), 'Other')
    )

print(balances['Account_Type'].value_counts())

# 8.0 Save Clean Balances
out_path = Path("data/processed/balances_clean.csv")
balances.to_csv(out_path, index=False)
print("Clean balances saved to:", out_path.resolve())

## 4.0 Locate Files

In [ ]:
print("cwd:", os.getcwd())

# Try both paths
p1 = Path("data/raw/balances")
p2 = Path("../data/raw/balances")

if p1.exists():
    bal_dir = p1
elif p2.exists():
    bal_dir = p2
else:
    raise FileNotFoundError("❌ Could not find balances folder. Check project structure.")

# Collect files
bal_files = list(bal_dir.glob("*.csv")) + list(bal_dir.glob("*.xlsx")) + list(bal_dir.glob("*.xls"))
print("Looking in:", bal_dir.resolve())
print("Files found:", len(bal_files))
for f in bal_files:
    print(" -", f.name)

if not bal_files:
    raise FileNotFoundError(f"No balance files found in {bal_dir}")/balances

## 5.0 Load and Merge Monthly Balances

In [ ]:
def read_file(f):
    if f.suffix.lower() in [".xlsx", ".xls"]:
        return pd.read_excel(f)
    return pd.read_csv(f)

balances = pd.concat([read_file(f) for f in bal_files], ignore_index=True)
balances.columns = [c.strip() for c in balances.columns]
print("Merged shape:", balances.shape)
balances.head()/balances

## 6.0 Clean Data
- Convert `date_opened` and other date fields.  
- Create `Reporting_Month`.  
- Drop duplicates if any.

In [ ]:
# Convert dates
if 'date_opened' in balances.columns:
    balances['date_opened'] = pd.to_datetime(balances['date_opened'], errors='coerce', dayfirst=True)

if 'birth_date' in balances.columns:
    balances['birth_date'] = pd.to_datetime(balances['birth_date'], errors='coerce', dayfirst=True)

if 'Reporting_Month' not in balances.columns and 'date_created' in balances.columns:
    balances['date_created'] = pd.to_datetime(balances['date_created'], errors='coerce', dayfirst=True)
    balances['Reporting_Month'] = balances['date_created'].dt.to_period('M').dt.to_timestamp('M')

balances.head()

## 7.0 Map GL Accounts → Account Type

In [ ]:
# Example mapping dictionary (adjust based on your GL mapping file)
gl_mapping = {
    'CURRENT ACCOUNT': 'Current',
    'SAVINGS ACCOUNT': 'Savings',
    'FIXED DEPOSIT': 'FD'
}

if 'gl_name' in balances.columns:
    balances['Account_Type'] = balances['gl_name'].map(lambda x: gl_mapping.get(str(x).upper(), 'Other'))

balances['Account_Type'].value_counts()/balances

## 8.0 Save Clean Balances

In [ ]:
out_path = Path("data/processed/balances_clean.csv")
balances.to_csv(out_path, index=False)
print("/balances Clean balances saved to:", out_path.resolve())

#  Next Steps
Proceed to **Dormancy_Aggregation.ipynb**:
- Merge transactions + balances.  
- Create customer×month dataset.  
- Add dormancy flags.  


---

# 03 — Aggregation: Transactions + Balances

## 1.0 Purpose
Now that we have clean **transactions** and **balances**, we merge them into one dataset.  
This creates a **customer × month panel dataset**, which will allow us to:  

- Track activity per customer, per month.  
- Define **dormancy** (no activity for X months).  
- Segment customers by account type (Current, Savings, FD).  

---

## 2.0 Data Sources
- `data/processed/transactions_clean.csv`  
- `data/processed/balances_clean.csv`  
- Merge on `cif_sub_no` (customer ID) and `Reporting_Month`.

In [ ]:
# 3.0 Import processed data
tx_clean = pd.read_csv("data/processed/transactions_clean.csv", parse_dates=["created_date","Reporting_Month"])
bal_clean = pd.read_csv("data/processed/balances_clean.csv", parse_dates=["Reporting_Month"])

print("Transactions:", tx_clean.shape)
print("Balances:", bal_clean.shape)

tx_clean.head(), bal_clean.head()/balances

## 4.0 Aggregate Transactions
We compute for each customer × month:
- **Total_Transactions**  
- **Total_Transaction_Value**  
- **Last_Transaction_Date**

In [ ]:
agg_tx = (
    tx_clean.groupby(["trs_ac_cif","Reporting_Month"])
    .agg(
        Total_Transactions=("trs_no","count"),
        Total_Transaction_Value=("amount","sum"),
        Last_Transaction_Date=("created_date","max")
    )
    .reset_index()
)

agg_tx.head()/balances

## 5.0 Merge with Balances
We join balances + transactions by `cif_sub_no` (customer ID) and `Reporting_Month`.  Full-time

In [ ]:
# Rename to align IDs
bal_clean = bal_clean.rename(columns={"cif_sub_no":"trs_ac_cif"})

df = bal_clean.merge(agg_tx, on=["trs_ac_cif","Reporting_Month"], how="left")

# Fill NaN for inactive customers (no transactions in that month)
df["Total_Transactions"] = df["Total_Transactions"].fillna(0)
df["Total_Transaction_Value"] = df["Total_Transaction_Value"].fillna(0)
df["Last_Transaction_Date"] = pd.to_datetime(df["Last_Transaction_Date"], errors="coerce")

print("Merged dataset:", df.shape)
df.head()

## 6.0 Define Dormancy Flag
Business rule:  
- **Dormant** = Customer has **0 transactions** in a month on a **Current Account**.  
- **Active** = At least 1 transaction.  

Later, we’ll refine this into multi-month dormancy windows.

In [ ]:
df["Dormant_Flag"] = ((df["Total_Transactions"] == 0) & (df["Account_Type"] == "Current")).astype(int)
df["Active_Flag"] = (df["Dormant_Flag"] == 0).astype(int)

df[["trs_ac_cif","Reporting_Month","Account_Type","Total_Transactions","Dormant_Flag"]].head(10)

## 7.0 Save Aggregated Dataset

In [ ]:
out_path = Path("data/processed/dormancy_dataset.csv")
df.to_csv(out_path, index=False)
print("Dormancy dataset saved to:", out_path.resolve())

#  Next Steps
Proceed to **04 — Dormancy Analysis**:
- Calculate dormancy trends per branch.  
- Track digital vs physical activity.  
- Identify high-risk customer segments.  


---

# 04 — Dormancy Analysis

## 1.0 Purpose
With the aggregated dataset prepared, we now analyze dormancy patterns:  

- Overall dormancy trends across months  
- Branch-level dormancy rates  
- Digital vs physical activity adoption  
- Segmentation by account type (Current, Savings, FD)

This will help identify high-risk segments and branches that require intervention.

In [ ]:
# 2.0 Import libraries
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

## 3.0 Load Dormancy Dataset

In [ ]:
df = pd.read_csv("data/processed/dormancy_dataset.csv", parse_dates=["Reporting_Month","Last_Transaction_Date"])
print("Shape:", df.shape)
df.head()

## 4.0 Overall Dormancy Trends
We measure the share of dormant customers per month.

In [ ]:
monthly_dormancy = (
    df.groupby("Reporting_Month")
    .agg(
        Dormant_Customers=("Dormant_Flag","sum"),
        Total_Customers=("trs_ac_cif","nunique")
    )
    .reset_index()
)
monthly_dormancy["Dormancy_Rate"] = monthly_dormancy["Dormant_Customers"] / monthly_dormancy["Total_Customers"]

monthly_dormancy

In [ ]:
plt.figure(figsize=(10,5))
sns.lineplot(data=monthly_dormancy, x="Reporting_Month", y="Dormancy_Rate", marker="o")
plt.title("Dormancy Rate Over Time")
plt.ylabel("Dormancy Rate (%)")
plt.xlabel("Month")
plt.xticks(rotation=45)
plt.show()

## 5.0 Branch-Level Dormancy
Which branches have the highest dormancy?

In [ ]:
branch_dormancy = (
    df.groupby(["Branch","Reporting_Month"])
    .agg(
        Dormant_Customers=("Dormant_Flag","sum"),
        Total_Customers=("trs_ac_cif","nunique")
    )
    .reset_index()
)
branch_dormancy["Dormancy_Rate"] = branch_dormancy["Dormant_Customers"] / branch_dormancy["Total_Customers"]

branch_summary = (
    branch_dormancy.groupby("Branch")
    .Dormancy_Rate.mean()
    .sort_values(ascending=False)
)
branch_summary.head(10)

In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(
    data=branch_summary.reset_index(),
    x="Dormancy_Rate",
    y="Branch"
)
plt.title("Average Dormancy Rate by Branch")
plt.xlabel("Dormancy Rate (%)")
plt.ylabel("Branch")
plt.show()

## 6.0 Digital vs Physical Activity
We compare activity in **digital channels (Mobile, Internet, Biz2Bank)** vs **physical (ATM)**.

In [ ]:
digital_summary = (
    df.groupby("Reporting_Month")
    .agg(
        Digital_Users=("IsDigital","sum"),
        Physical_Users=("IsPhysical","sum"),
        Total_Customers=("trs_ac_cif","nunique")
    )
    .reset_index()
)

digital_summary

In [ ]:
plt.figure(figsize=(10,5))
sns.lineplot(data=digital_summary, x="Reporting_Month", y="Digital_Users", label="Digital")
sns.lineplot(data=digital_summary, x="Reporting_Month", y="Physical_Users", label="Physical")
plt.title("Digital vs Physical Channel Activity")
plt.ylabel("Number of Customers")
plt.xlabel("Month")
plt.xticks(rotation=45)
plt.legend()
plt.show()

## 7.0 Account Type Segmentation
We check dormancy rates across Current, Savings, and FD.

In [ ]:
acct_dormancy = (
    df.groupby(["Account_Type","Reporting_Month"])
    .agg(
        Dormant_Customers=("Dormant_Flag","sum"),
        Total_Customers=("trs_ac_cif","nunique")
    )
    .reset_index()
)
acct_dormancy["Dormancy_Rate"] = acct_dormancy["Dormant_Customers"] / acct_dormancy["Total_Customers"]

plt.figure(figsize=(10,5))
sns.lineplot(data=acct_dormancy, x="Reporting_Month", y="Dormancy_Rate", hue="Account_Type", marker="o")
plt.title("Dormancy Rate by Account Type")
plt.ylabel("Dormancy Rate (%)")
plt.xlabel("Month")
plt.xticks(rotation=45)
plt.show()

# Next Steps
- Prepare **findings & recommendations** based on trends.  
- Design **branch dashboards** to monitor dormancy.  
- Feed results into a final **report for management**.
